In [29]:
import os
import pandas as pd
import numpy as np
import nipy
from mvpa2.suite import *

In [30]:
def load_data(subject_number):
    """Returns an fmri dataset object containing data of all 10 runs for a particular subject."""
    
    cwd = '/home/fmri-data'
    data_path = os.path.join(cwd,"mvpa_"+str(subject_number))

    # Load the sample attributes
    attr_fname = "targets_attr_mvpa_"+str(subject_number)+".txt"
    attr_path = os.path.join(data_path, attr_fname)
    attr = SampleAttributes(attr_path) # PyMVPA convenience function to read attribute files.

    # Load all 10 runs from one subject into one fMRI dataset
    file_path_list = list() # file path list to each run
    for i in range(1,11):
        run_name = "run"+str(i)
        fname = run_name +"_native.nii.gz"
        filepath = os.path.join(data_path, fname)
        file_path_list.append(filepath)

    # fMRI dataset format is convenient and vital for using PyMVPA.
    ds = fmri_dataset(file_path_list,targets=attr.targets,chunks=attr.chunks)
    return ds
if __debug__:
    debug.active += ["SLC"]

ds = load_data(1)
first_ds = ds
poly_detrend(ds, polyord=1, chunks_attr='chunks')
zscore(ds, param_est=('targets', ['baseline_catch_all']), chunks_attr='chunks')
ds_dropped = ds[ds.sa.targets != 'baseline_catch_all']

In [37]:
clf = FeatureSelectionClassifier(SMLR(), 
                                 SensitivityBasedFeatureSelection(
                                     SMLRWeights(SMLR(lm=1.0), 
                                                 postproc=sumofabs_sample()),
                                     FixedNElementTailSelector(1000, 
                                                               tail='upper', 
                                                               mode='select')))  
# clf = SMLRWeights(SMLR(), force_train=True)
clf.train(ds_dropped)

In [38]:
selected_voxels = clf.mapper.forward(ds_dropped)
ds_dropped.fa.voxel_indices

array([[ 0,  0,  0],
       [ 0,  0,  1],
       [ 0,  0,  2],
       ...,
       [73, 73, 30],
       [73, 73, 31],
       [73, 73, 32]])

In [39]:
def flatten_index(voxel_index):
    return voxel_index[0] * 74 * 33 + voxel_index[1] * 33 + voxel_index[2]
important_flatindexes = []
for indice in selected_voxels.fa.voxel_indices:
    important_flatindexes.append(flatten_index(indice))

In [40]:
# important_flatindexes

In [41]:
count = 0
# for volume in ds.samples:
for i, vox in enumerate(ds_dropped.samples[150]):
    if i in important_flatindexes:
        vox = 10000
    else:
        vox = 0
    count += 1
    if count % 10000 == 0:
        print count, ' volume done'

10000  volume done
20000  volume done
30000  volume done
40000  volume done
50000  volume done
60000  volume done
70000  volume done
80000  volume done
90000  volume done
100000  volume done
110000  volume done
120000  volume done
130000  volume done
140000  volume done
150000  volume done
160000  volume done
170000  volume done
180000  volume done


* map sensitivities back to nifti image

In [ ]:
fds = clf.mapper.forward(ds)
mynii_fds = map2nifti(fds, ds.samples)
mynii_fds.to_filename('ec2_1k_zero.nii.gz')